<a href="https://colab.research.google.com/github/yunhsuan1009/yunhsuan/blob/main/EX05_03_Reply_Multi_type_Messagesipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 14.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
  ngrok.set_auth_token(userdata.get('ngrok'))
  ngrok.connect(5000)
  run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)
app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('Channel_access_token'))

handler = WebhookHandler(userdata.get('secrettalk'))

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)


    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:

        abort(400)

    return 'OK'

import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel("gemini-2.0-flash")
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action =='sticker':

          reply = StickerMessage(
             package_id='446',
             sticker_id='1989'
          )
        elif action == 'image':
          reply = ImageMessage(
            original_content_url='https://image.kkday.com/v2/image/get/s1.kkday.com/campaign_863/20190813095241_xiRDy/jpg'
            ,preview_image_url='https://image.kkday.com/v2/image/get/s1.kkday.com/campaign_863/20190813095241_xiRDy/jpg'
          )
        elif action == 'video':
          reply = VideoMessage(
            original_content_url='https://cdn.pixabay.com/video/2021/07/08/80724-573496156_large.mp4'
            ,preview_image_url='https://www.leo-travel.idv.tw/wp-content/uploads/austria-vuenna-inner-city-shutterstock_545763607.jpg'
          )
        elif action == 'location':
          reply = LocationMessage(
            title='Taipei101',
            address='110台北市信義區信義路五段7號',
            latitude=25.047924
            ,longitude=121.517081
          )
        else:
          response = ask_gemini(action)
          reply = TextMessage(text=response)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )
ngrok_start()
if __name__ == "__main__":
  app.run()




 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5373-34-169-208-132.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 09:49:56] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 09:49:57] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 09:50:12] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 09:51:37] "POST /callback HTTP/1.1" 200 -
